In [121]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [122]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import pymysql
from pygeocoder import Geocoder
import geocoder
import reverse_geocoder as rg

In [200]:
ny_crime_data = "data/NYPD_Complaint_Data_Current__Year_To_Date_.csv"
home_value = "data/Zip_Zhvi_Summary_AllHomes.csv"
zip_db = "data/zip_code_database.csv"

In [201]:
ny_data = pd.read_csv(ny_crime_data, encoding="iso-8859-1")
zip_data = pd.read_csv(home_value, encoding="iso-8859-1")
zip_db = pd.read_csv(zip_db)

In [202]:

zip_db = zip_db.rename(columns={"zip": "Zipcode"})
zip_db = zip_db.merge(NY_Zip, on="Zipcode", how="inner")
zip_db.head()
zip_db = pd.DataFrame({"Zipcode": zip_db["Zipcode"],"Latitude": zip_db["latitude"], "Longitude": zip_db["longitude"]})


In [203]:
zip_db.head()

,Zipcode,Latitude,Longitude
0,10001,40.75,-74.00
1,10002,40.71,-73.99
2,10003,40.73,-73.99
3,10004,40.69,-74.02
4,10005,40.71,-74.01


In [204]:
NY_Zip_Codes = "10001, 10002, 10003, 10004, 10005, 10006, 10007, 10009, 10010, 10011, 10012, 10013, 10014, 10016, 10017, 10018, 10019, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 10040, 10044, 10065, 10069, 10075, 10103, 10110, 10111, 10112, 10115, 10119, 10128, 10152, 10153, 10154, 10162, 10165, 10167, 10168, 10169, 10170, 10171, 10172, 10173, 10174, 10177, 10199, 10271, 10278, 10279, 10280, 10282, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10311, 10312, 10314, 10451, 10452, 10453, 10454, 10455, 10456, 10457, 10458, 10459, 10460, 10461, 10462, 10463, 10464, 10465, 10466, 10467, 10468, 10469, 10470, 10471, 10472, 10473, 10474, 10475, 11001, 11003, 11004, 11005, 11040, 11101, 11102, 11103, 11104, 11105, 11106, 11109, 11201, 11203, 11204, 11205, 11206, 11207, 11208, 11209, 11210, 11211, 11212, 11213, 11214, 11215, 11216, 11217, 11218, 11219, 11220, 11221, 11222, 11223, 11224, 11225, 11226, 11228, 11229, 11230, 11231, 11232, 11233, 11234, 11235, 11236, 11237, 11238, 11239, 11351, 11354, 11355, 11356, 11357, 11358, 11359, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11374, 11375, 11377, 11378, 11379, 11385, 11411, 11412, 11413, 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421, 11422, 11423, 11424, 11425, 11426, 11427, 11428, 11429, 11430, 11432, 11433, 11434, 11435, 11436, 11451, 11691, 11692, 11693, 11694, 11697"

NY_Zip_Codes = NY_Zip_Codes.split(", ")
NY_Zip_Codes = list(map(int, NY_Zip_Codes))
NY_Zip = pd.DataFrame({"Zipcode": NY_Zip_Codes})

In [205]:
zip_data = zip_data.rename(columns={"RegionName": "Zipcode"})
zip_data = zip_data.merge(NY_Zip, on="Zipcode", how="inner")
zip_data.head()

,Date,RegionID,Zipcode,State,Metro,County,City,SizeRank,Zhvi,MoM,QoQ,YoY,5Year,10Year,PeakMonth,PeakQuarter,PeakZHVI,PctFallFromPeak,LastTimeAtCurrZHVI
0,2018-10-31,61639,10025,NY,New York-Newark-Jersey City,New York County,New York,0,1080800,-0.003963,-0.019683,-0.036634,NaN,NaN,2017-12,2017-Q4,1138600,-0.050764,2017-06
1,2018-10-31,61637,10023,NY,New York-Newark-Jersey City,New York County,New York,2,1398800,-0.013331,-0.047658,-0.124765,NaN,NaN,2016-12,2016-Q4,1618100,-0.135529,2014-04
2,2018-10-31,61616,10002,NY,New York-Newark-Jersey City,New York County,New York,8,985200,-0.011538,-0.021162,-0.024554,NaN,NaN,2017-10,2017-Q4,1010000,-0.024554,2017-06
3,2018-10-31,61807,10467,NY,New York-Newark-Jersey City,Bronx County,New York,9,424800,0.006158,0.026583,0.128587,NaN,NaN,2018-10,2018-Q4,424800,0.000000,2018-10
4,2018-10-31,62037,11226,NY,New York-Newark-Jersey City,Kings County,New York,12,824300,-0.001091,0.003408,0.184509,NaN,NaN,2018-09,2018-Q3,825200,-0.001091,2018-08


In [206]:
zip_db = zip_db.merge(zip_data, on="Zipcode", how="inner")
zip_db = pd.DataFrame({"Zipcode": zip_db["Zipcode"],"Latitude": zip_db["Latitude"], "Longitude": zip_db["Longitude"], "Zhvi": zip_db["Zhvi"]})
zip_db.head()

,Zipcode,Latitude,Longitude,Zhvi
0,10001,40.75,-74.00,2003600
1,10002,40.71,-73.99,985200
2,10003,40.73,-73.99,1666000
3,10004,40.69,-74.02,1113700
4,10005,40.71,-74.01,1164600


In [207]:
ny_data = ny_data.merge(NY_Zip, on="Zipcode", how="inner")
ny_data

,CMPLNT_NUM,ADDR_PCT_CD,State,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,...,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,Adress,Unnamed: 37,Zipcode
0,457872064,113,New York,QUEENS,6/30/2018,23:30:00,6/30/2018,23:40:00,COMPLETED,NaN,...,BLACK,F,1048111,188479,40.683786,-73.769745,"(40.68378593, -73.769745503)","Merrick Blvd + Baisley Blvd, Queens, NY 11434,...","Merrick Blvd + Baisley Blvd, Queens, NY 11434,",11434
1,443852823,113,New York,QUEENS,6/30/2018,21:30:00,6/30/2018,21:45:00,COMPLETED,NaN,...,BLACK,M,1048403,185653,40.676027,-73.768720,"(40.676027116, -73.76871958)","172-44 133rd Ave, Jamaica, NY 11434, USA","172-44 133rd Ave, Jamaica, NY 11434,",11434
2,522843547,113,New York,QUEENS,6/30/2018,21:30:00,6/30/2018,21:45:00,COMPLETED,NaN,...,BLACK,M,1048403,185653,40.676027,-73.768720,"(40.676027116, -73.76871958)","172-44 133rd Ave, Jamaica, NY 11434, USA","172-44 133rd Ave, Jamaica, NY 11434,",11434
3,504721590,113,New York,QUEENS,6/30/2018,21:20:00,6/30/2018,21:30:00,COMPLETED,NaN,...,BLACK,M,1050371,187013,40.679746,-73.761611,"(40.679745514, -73.761611316)","Farmers Bl/Merrick Bl, Queens, NY 11434, USA","Farmers Bl/Merrick Bl, Queens, NY 11434,",11434
4,497195999,113,New York,QUEENS,6/30/2018,16:30:00,6/30/2018,16:35:00,COMPLETED,NaN,...,BLACK,F,1049010,182682,40.667868,-73.766560,"(40.667867993, -73.766559801)","144-20 Farmers Blvd, Jamaica, NY 11434, USA","144-20 Farmers Blvd, Jamaica, NY 11434,",11434
5,409549163,113,New York,QUEENS,6/30/2018,16:20:00,6/30/2018,16:25:00,ATTEMPTED,NaN,...,BLACK,F,1048403,182768,40.668108,-73.768747,"(40.668108463, -73.768747038)","169-57 144th Rd, Jamaica, NY 11434, USA","169-57 144th Rd, Jamaica, NY 11434,",11434
6,478394187,113,New York,QUEENS,6/30/2018,15:40:00,6/30/2018,15:50:00,COMPLETED,NaN,...,BLACK,M,1047059,189285,40.686006,-73.773531,"(40.686005737, -73.773531042)","118-44 Merrick Blvd, Jamaica, NY 11434, USA","118-44 Merrick Blvd, Jamaica, NY 11434,",11434
7,673297413,113,New York,QUEENS,6/30/2018,13:10:00,6/30/2018,13:20:00,COMPLETED,NaN,...,UNKNOWN,M,1044526,183558,40.670304,-73.782715,"(40.670304074, -73.782715463)","153-21 Rockaway Blvd, Jamaica, NY 11434, USA","153-21 Rockaway Blvd, Jamaica, NY 11434,",11434
8,981016979,113,New York,QUEENS,6/30/2018,2:45:00,NaN,NaN,COMPLETED,NaN,...,BLACK,F,1050153,187120,40.680041,-73.762396,"(40.68004083, -73.762396234)","130-24 Merrick Blvd, Jamaica, NY 11434, USA","130-24 Merrick Blvd, Jamaica, NY 11434,",11434
9,618658052,113,New York,QUEENS,6/29/2018,15:30:00,6/30/2018,6:00:00,COMPLETED,NaN,...,BLACK,M,1048142,192668,40.695284,-73.769594,"(40.69528351, -73.769594002)","114-32 180th St, Jamaica, NY 11434, USA","114-32 180th St, Jamaica, NY 11434,",11434


In [208]:
ny_data = ny_data.groupby("Zipcode").count()

In [209]:
ny_data = pd.DataFrame({"Number of Crimes": ny_data["State"]})
ny_data.head()

,Number of Crimes
Zipcode,
10001,201
10002,175
10003,176
10004,19
10005,22


In [210]:
ny_data= ny_data.reset_index()
zip_db = zip_db.merge(ny_data, on="Zipcode", how="inner")


,Zipcode,Latitude,Longitude,Zhvi,Number of Crimes
0,10001,40.75,-74.00,2003600,201
1,10002,40.71,-73.99,985200,175
2,10003,40.73,-73.99,1666000,176
3,10004,40.69,-74.02,1113700,19
4,10005,40.71,-74.01,1164600,22
5,10006,40.71,-74.01,943800,6
6,10007,40.71,-74.01,2711900,45
7,10009,40.73,-73.98,1151900,101
8,10010,40.74,-73.98,1718700,81
9,10011,40.74,-74.00,1675900,128


In [211]:
zip_db.to_csv("Ny_Data.csv")